# TechCraft Gaming PC Production Optimization

## Problem Statement
TechCraft builds custom gaming PCs for North American customers. Their flagship product is a high-end gaming rig that customers can customize with different GPUs, cooling systems, and RGB lighting.

The operations team needs to determine the optimal production plan for next month across their facilities:
- Primary facilities:
  * Taiwan facility (T1)
  * Mexico facility (M1)
  * California facility (C)
- Optional facilities that can be repurposed:
  * Taiwan second line (T2)
  * Mexico second line (M2)

### Key Constraints:
- All primary production lines must be used
- Only one additional line can be repurposed (due to quality control limitations)
- Demand must be met exactly (no partial units)
- Distribution to two warehouses: Las Vegas (WH1) and Denver (WH2)

### Expected Demand:
- Las Vegas (WH1): 5,500 units
- Denver (WH2): 4,800 units

### Profit per Unit ($):
| Production Line | Las Vegas (WH1) | Denver (WH2) |
|----------------|-----------------|---------------|
| Taiwan (T1)    | 450             | 380          |
| Mexico (M1)    | 420             | 400          |
| California (C) | 380             | 350          |
| Taiwan 2 (T2)  | 400             | 340          |
| Mexico 2 (M2)  | 380             | 360          |

---

In [ ]:
# Required package installations (uncomment if needed)
# !pip install pulp
# !pip install numpy
# !pip install pandas

import pulp
import numpy as np
import pandas as pd

## Step 1: Define the Linear Programming Problem

In [ ]:
# Create the optimization model
model = pulp.LpProblem("TechCraft_Production_Optimization", pulp.LpMaximize)

# Define production lines and warehouses
production_lines = ['T1', 'M1', 'C', 'T2', 'M2']
warehouses = ['WH1', 'WH2']

# Define profits
profits = {
    ('T1', 'WH1'): 450, ('T1', 'WH2'): 380,
    ('M1', 'WH1'): 420, ('M1', 'WH2'): 400,
    ('C', 'WH1'): 380, ('C', 'WH2'): 350,
    ('T2', 'WH1'): 400, ('T2', 'WH2'): 340,
    ('M2', 'WH1'): 380, ('M2', 'WH2'): 360
}

# Decision variables
vars = pulp.LpVariable.dicts("Units",
                            ((i, j) for i in production_lines for j in warehouses),
                            lowBound=0,
                            cat='Integer')

# Binary variables for optional production lines
use_line = pulp.LpVariable.dicts("UseLine",
                                ['T2', 'M2'],
                                cat='Binary')

# Objective function
model += pulp.lpSum(vars[i,j] * profits[i,j] for i in production_lines for j in warehouses)

# Demand constraints
model += pulp.lpSum(vars[i,'WH1'] for i in production_lines) == 5500  # Las Vegas demand
model += pulp.lpSum(vars[i,'WH2'] for i in production_lines) == 4800  # Denver demand

# Only one optional line can be used
model += use_line['T2'] + use_line['M2'] <= 1

# Link optional lines to their usage
M = 10000  # Big M constant
for j in warehouses:
    model += vars['T2',j] <= M * use_line['T2']
    model += vars['M2',j] <= M * use_line['M2']

# Solve the model
model.solve()

print(f"Status: {pulp.LpStatus[model.status]}")

## Step 2: Analyze Results

In [ ]:
# Create results dataframe
results = []
for i in production_lines:
    for j in warehouses:
        if vars[i,j].value() > 0:
            results.append({
                'Production Line': i,
                'Warehouse': j,
                'Units': int(vars[i,j].value()),
                'Profit per Unit': profits[i,j],
                'Total Profit': int(vars[i,j].value() * profits[i,j])
            })

df_results = pd.DataFrame(results)
print("Production Plan:")
print(df_results)

print(f"\nTotal Profit: ${pulp.value(model.objective):,.2f}")

# Show which optional line was used
print("\nOptional Lines Used:")
for line in ['T2', 'M2']:
    if use_line[line].value() == 1:
        print(f"- {line} is active")

## Step 3: Verify Constraints

In [ ]:
# Verify warehouse demands are met
wh_totals = df_results.groupby('Warehouse')['Units'].sum()
print("Warehouse Demand Verification:")
print(f"WH1 (Las Vegas): {wh_totals['WH1']:,} units (Target: 5,500)")
print(f"WH2 (Denver): {wh_totals['WH2']:,} units (Target: 4,800)")

# Verify optional line constraint
optional_lines_used = sum(use_line[line].value() for line in ['T2', 'M2'])
print(f"\nOptional lines used: {optional_lines_used} (Maximum: 1)")

## Conclusions

The optimization model has determined:

1. **Optimal Production Plan:** [Will be filled after running]
2. **Maximum Profit:** [Will be filled after running]
3. **Optional Line Usage:** [Will be filled after running]

This solution:
- Meets exact demand for both warehouses
- Uses primary production lines efficiently
- Respects the constraint of using at most one optional line
- Maximizes total profit while satisfying all constraints